Tracy Huynh    
DA320 
Midterm Assignment 

In [6]:
# Importing all Modules/libraries needed
import re
import urllib3
import certifi
import json
import pymongo
import pandas

In [7]:
# Constructing REGEX expressions
title_regex = re.compile(r"class=\"title\"><h3>(.+)</h3>")
date_regex = re.compile(r"class=\"clamp-details\">\s+<span>(.+)</span>")
description_regex = re.compile(r"<div class=\"summary\">\s*([\S\s]+?)\s*</div>")
score_regex = re.compile(r"<span class=\"title\">Metascore:</span>\s+<a class=\"metascore_anchor\" href=\"/movie/.*?/critic-reviews\">\s+<div class=\"metascore_w large movie .+\">(.*?)</div>")
image_regex = re.compile(r"<a href=\"/movie/.*\"><img src=\"(.*)\" alt=\"")

# Construct an HTTP Pool for Connections
http = urllib3.PoolManager(ca_certs=certifi.where())

In [8]:
# Retrieve Credentials for SQL Server from files 
with open('/Users/misstacyhuynh/Desktop/Fall 2022/DA320/Secretconnectionstring.json') as f:
    data = json.load(f)
    mongo_connection_string = data['mongodb']

# Fetching the database 
client = pymongo.MongoClient(mongo_connection_string, tlsCAFile=certifi.where())
da320_database = client ['Movies']
metacritic_data = da320_database['metacritic']

In [9]:
# retrieve list of movies from a year and page
def metacritic_scraper(year: int, page: int) -> pandas.DataFrame:

    url = f"https://www.metacritic.com/browse/movies/score/metascore/year/filtered?year_selected={year}&sort=desc&view=detailed&page={page}"
    response = http.request('GET', url, headers={'User-Agent': 'Mozilla/5.0'})
    datastring = str(response.data, "utf-8")

    # Execute expressions 
    titles = title_regex.findall(datastring)
    dates = date_regex.findall(datastring)
    descriptions = description_regex.findall(datastring)
    scores = score_regex.findall(datastring)
    images = image_regex.findall(datastring)

    # return collection
    dataset = {"title": titles, "date": dates, "description": descriptions, "score": scores, "image": images}
    return pandas.DataFrame(dataset)

In [10]:
# loop through Years and Pages

import time

for year in range(2000, 2023): 
    page = 0 
    print(f"Collecting data for {year} page {page}...")

    while True:
        data = metacritic_scraper(year, page)

        if len(data) == 0: 
            break

        movies_to_insert = []
        for row in data.itertuples(): 
            movie = {
                "title": row.title,
                "release_date": row.date, 
                "description": row.description,
                "meta_score": row.score,
                "image": row.image,
        }

        movies_to_insert.append(movie)

        # Inserting records into mongodb
        print(f"Inserting {len(movies_to_insert)} movies for the year {year} page {page}")
        metacritic_data.insert_many(movies_to_insert)
        page = page + 1

Inserting 1 movies for the year 2000 page 0
Inserting 1 movies for the year 2000 page 1
Inserting 1 movies for the year 2000 page 2
Inserting 1 movies for the year 2000 page 3
Inserting 1 movies for the year 2001 page 0
Inserting 1 movies for the year 2001 page 1
Inserting 1 movies for the year 2001 page 2
Inserting 1 movies for the year 2001 page 3
Inserting 1 movies for the year 2002 page 0
Inserting 1 movies for the year 2002 page 1
Inserting 1 movies for the year 2002 page 2
Inserting 1 movies for the year 2002 page 3
Inserting 1 movies for the year 2002 page 4
Inserting 1 movies for the year 2003 page 0
Inserting 1 movies for the year 2003 page 1
Inserting 1 movies for the year 2003 page 2
Inserting 1 movies for the year 2003 page 3
Inserting 1 movies for the year 2003 page 4
Inserting 1 movies for the year 2004 page 0
Inserting 1 movies for the year 2004 page 1
Inserting 1 movies for the year 2004 page 2
Inserting 1 movies for the year 2004 page 3
Inserting 1 movies for the year 